In [35]:
import ast
import pandas as pd

# Fájl elérési útja
file_path_alapanyagok = r"C:\Users\Bálint\Desktop\Asztal\Projekt\alapanyagok_jav.csv"
file_path_receptek = r"C:\Users\Bálint\Desktop\Asztal\Projekt\receptek.csv"

# Fájl beolvasása DataFrame-ként
df_alapanyagok = pd.read_csv(file_path_alapanyagok, encoding='utf-8')
df_receptek = pd.read_csv(file_path_receptek, encoding='utf-8')
#Converting the wrongly scraped ingredients column to a dictionary using ast
df_receptek["Hozzávalók"] = df_receptek["Hozzávalók"].apply(ast.literal_eval)

In [37]:
import pandas as pd
import re

class DataCleaner:
    """Class for cleaning and processing nutritional data."""

    def __init__(self, df=None):
        """Initialize with a DataFrame if provided."""
        self.df = df

    @staticmethod
    def convert_measures(df):
        """Convert µg values to grams for selected nutrient columns."""
        columns_to_convert = [
            "Folát", "Béta-karotin", "Lutein+zeaxantin", "K vitamin", "A vitamin", 
            "Likopin", "Alfa-karotin", "Béta-kriptoxantin", "Retinol", "Folsav", "Szelén"
        ]
        for col in columns_to_convert:
            if col in df.columns:
                df[col] = df[col] / 1_000_000  # Convert µg to g
        return df

    @staticmethod
    def detect_outliers(df, threshold=1000):
        """Detect unusually high values in numeric columns."""
        exceptions = {
            "tojássárgája": "koleszterin",
            "sertés agyvelő": "koleszterin",
            "sertésvelő": "koleszterin",
            "marhavelő": "koleszterin",
            "szárított rák": "koleszterin"
        }

        for col in df.columns:
            if col != "Alapanyag neve":
                df[col] = pd.to_numeric(df[col], errors="coerce")

        outlier_rows = []
        for col in df.columns:
            if col != "Alapanyag neve":
                high_values = df[df[col] > threshold]

                for _, row in high_values.iterrows():
                    if row["Alapanyag neve"] in exceptions and exceptions[row["Alapanyag neve"]] == col:
                        continue  # Skip known exceptions

                    outlier_rows.append({
                        "Alapanyag neve": row["Alapanyag neve"],
                        "Tápanyag": col,
                        "Érték": row[col]
                    })
        
        return pd.DataFrame(outlier_rows)

    @staticmethod
    def extract_unit(amount):
        """Extracts the unit from a quantity string."""
        match = re.search(r'[^\d\s,.]+$', amount)  # Extract last non-numeric part
        return match.group() if match else None

    def get_unique_units(self, column="Hozzávalók"):
        """Find unique measurement units from the ingredients column."""
        unique_units = set()
        for ingredients in self.df[column]:
            for ingredient in ingredients:
                unit = self.extract_unit(ingredient["Mennyiség"])
                if unit:
                    unique_units.add(unit)
        return unique_units

    @staticmethod
    def convert_to_100g(amount, unit_to_grams):
        """Convert ingredient amounts to a proportion of 100g."""
        num_match = re.search(r"[-+]?\d*\.?\d+", amount)
        unit_match = re.search(r'[^\d\s,.]+$', amount)
        
        unit_to_grams = {
            "g": 1,             # 1 gram = 1 gram
            "dkg": 10,          # 1 dkg = 10 grams
            "kg": 1000,         # 1 kg = 1000 grams
            "ml": 1,            # 1 ml ≈ 1 gram (for water-based liquids)
            "l": 1000,          # 1 liter = 1000 grams
            "dl": 100,          # 1 dl ≈ 100 grams
            "tk": 5,            # 1 teáskanál (teaspoon) ≈ 5 grams
            "kk": 3,            # 1 kávéskanál (coffee spoon) ≈ 3 grams
            "ek": 10,           # 1 evőkanál (tablespoon) ≈ 10 grams
            "teáskanál": 5,     # Same as "tk" (teaspoon)
            "evőkanál": 10,     # Same as "ek" (tablespoon)
            "bögre": 250,       # 1 bögre (cup) ≈ 250 grams (varies by ingredient)
            "csésze": 200,      # 1 csésze (cup) ≈ 200 grams (varies by ingredient)
            "csipet": 1,        # 1 csipet = small pinch ≈ 1 gram
            "késhegynyi": 2,    # 1 késhegynyi (knife tip amount) ≈ 2 grams
            "csepp": 0.05,      # 1 csepp (drop) ≈ 0.05 grams
            "tasak": 10,        # 1 tasak (packet) ≈ 10 grams (common for yeast, spices)
            "doboz": 400,       # 1 doboz (box) ≈ 400 grams (e.g., canned food)
            "csomag": 500,      # 1 csomag (pack) ≈ 500 grams (e.g., pasta, flour)
            "tubus": 100,       # 1 tubus (tube) ≈ 100 grams (e.g., tomato paste)
            "szál": 15,         # 1 szál (stalk) ≈ 15 grams (e.g., green onion, celery)
            "szelet": 30,       # 1 szelet (slice) ≈ 30 grams (e.g., bread, cheese)
            "szem": 5,          # 1 szem (piece) ≈ 5 grams (e.g., nut, seed, grape)
            "gerezd": 5,        # 1 gerezd (clove) ≈ 5 grams (e.g., garlic)
            "fej": 15,         # 1 fej (head) ≈ 200 grams (e.g., onion, garlic, cabbage)
            "csokor": 10,       # 1 csokor (bunch) ≈ 50 grams (e.g., parsley, dill)
            "levél": 1,         # 1 levél (leaf) ≈ 1 gram (e.g., basil, lettuce)
            "hüvelykujjnyi": 3,# 1 hüvelykujjnyi (thumb-sized) ≈ 25 grams (e.g., butter)
            "marék": 25,        # 1 marék (handful) ≈ 30 grams
            "maréknyi": 25,     # Same as "marék" (handful)
            "szár": 10,         # 1 szár (stalk) ≈ 15 grams
            "üveg": 500,        # 1 üveg (bottle) ≈ 500 grams (e.g., honey, oil)
            "szerint": None,    # "Ízlés szerint" (to taste) cannot be converted
            "nyert": None,      # Unknown meaning, needs manual adjustment
            "gr)": None         # Possible typo for "g"
        }

        if not num_match or not unit_match:
            return None  # If missing number or unit, return None

        num = float(num_match.group())
        unit = unit_match.group()

        if unit == "db":
            return 1  # Default proportion for single items

        if unit in unit_to_grams and unit_to_grams[unit]:
            return (num * unit_to_grams[unit]) / 100  # Normalize to 100g
        else:
            return None  # Unknown or unconvertible unit


In [133]:
def calculate_recipe_nutrition(df_receptek, df_alapanyagok):
    # Convert nutritional values to float to prevent TypeError
    for col in df_alapanyagok.columns:
        if col != 'Alapanyag neve':  
            df_alapanyagok[col] = pd.to_numeric(df_alapanyagok[col], errors='coerce')

    # Create a fast lookup set of ingredient names
    alapanyagok_set = set(df_alapanyagok['Alapanyag neve'])
    mismatching_ingredients = []  
    log_entries = []  
    calculated_nutrition = []

    # Iterate over only the first recipe
    row = df_receptek.iloc[0]  # Select only the first row
    recipe_name = row['Recept neve']
    ingredients_list = row['Hozzávalók']  # List of dictionaries

    # Dictionary to store aggregated nutrition values
    aggregated_nutrition = {col: 0 for col in df_alapanyagok.columns if col != 'Alapanyag neve'}

    # Iterate over ingredients
    for ingredient in ingredients_list:
        name = ingredient['Hozzávaló']
        amount = ingredient['Mennyiség']

        # Convert amount to proportion of 100g
        proportion_100g = convert_to_100g(amount)
        ingredient["Proportion_100g"] = proportion_100g

        if proportion_100g is None:
            mismatching_ingredients.append(name)
            continue

        # Check if ingredient exists in nutritional data
        if name not in alapanyagok_set:
            mismatching_ingredients.append(name)
            continue

        # Get nutritional values from df_alapanyagok
        matching_ingredient = df_alapanyagok[df_alapanyagok['Alapanyag neve'] == name]
        ingredient_nutrition = {col: 0 for col in df_alapanyagok.columns if col != 'Alapanyag neve'}

        for col in ingredient_nutrition.keys():
            if col in df_alapanyagok.columns:
                value = matching_ingredient.iloc[0][col]
                if pd.notna(value):  # Ensure it's a number
                    ingredient_nutrition[col] = float(value) * proportion_100g
        
        # Add ingredient-level nutrition to total
        for col in aggregated_nutrition.keys():
            aggregated_nutrition[col] += ingredient_nutrition[col]
    
    calculated_nutrition.append(aggregated_nutrition)

#     # Compare original and calculated values
#     log_entry = []
#     for col, calculated_value in aggregated_nutrition.items():
#         original_value = row.get(col, np.nan)  # Get original value or NaN

#         if not (pd.isna(original_value) and pd.isna(calculated_value)) and not np.isclose(original_value, calculated_value, atol=0.01):
#             log_entry.append({
#                 'Recept neve': recipe_name,
#                 'Eltérés': col,
#                 'Eredeti érték': original_value,
#                 'Számolt érték': calculated_value
#             })

#     # Save log if there are discrepancies
#     if log_entry:
#         log_df = pd.DataFrame(log_entry)
#         log_df.to_csv(f'{recipe_name}_recalculated_nutrition.csv', index=False)

#     # Add the calculated nutrition values to the DataFrame
#     df_receptek.at[0, 'Számolt tápértékek'] = aggregated_nutrition

#     # Print missing ingredients
#     print("Hiányzó hozzávalók listája:", set(mismatching_ingredients))

    return df_receptek


In [124]:
# Running the function
df_receptek = calculate_recipe_nutrition(df_receptek, df_alapanyagok)

In [131]:
df_filtered = df_receptek[df_receptek["Recept neve"] == "Csavaros lasagne"]
df_filtered.to_csv("filtered_recept.csv", index=False)

In [137]:
for col in df_alapanyagok.columns:
    if col != 'Alapanyag neve':  
        df_alapanyagok[col] = pd.to_numeric(df_alapanyagok[col], errors='coerce')

# Create a fast lookup set of ingredient names
alapanyagok_set = set(df_alapanyagok['Alapanyag neve'])
mismatching_ingredients = []  
log_entries = []  
calculated_nutrition = []

# Iterate over only the first recipe
row = df_receptek.iloc[0]  # Select only the first row
recipe_name = row['Recept neve']
ingredients_list = row['Hozzávalók']  # List of dictionaries

# Dictionary to store aggregated nutrition values
aggregated_nutrition = {col: 0 for col in df_alapanyagok.columns if col != 'Alapanyag neve'}

# Iterate over ingredients
for ingredient in ingredients_list:
    name = ingredient['Hozzávaló']
    amount = ingredient['Mennyiség']

    # Convert amount to proportion of 100g
    proportion_100g = convert_to_100g(amount)
    ingredient["Proportion_100g"] = proportion_100g

    if proportion_100g is None:
        mismatching_ingredients.append(name)
        continue

    # Check if ingredient exists in nutritional data
    if name not in alapanyagok_set:
        mismatching_ingredients.append(name)
        continue

    # Get nutritional values from df_alapanyagok
    matching_ingredient = df_alapanyagok[df_alapanyagok['Alapanyag neve'] == name]
    ingredient_nutrition = {col: 0 for col in df_alapanyagok.columns if col != 'Alapanyag neve'}

    for col in ingredient_nutrition.keys():
        if col in df_alapanyagok.columns:
            value = matching_ingredient.iloc[0][col]
            if pd.notna(value):  # Ensure it's a number
                ingredient_nutrition[col] = float(value) * proportion_100g

    # Debug print for ingredient-level calculations
    print(f"Ingredient: {name}, Proportion_100g: {proportion_100g}, Nutrition: {ingredient_nutrition}")

    # Add ingredient-level nutrition to total
    for col in aggregated_nutrition.keys():
        aggregated_nutrition[col] += ingredient_nutrition[col]

# # Debug print for final aggregated nutrition
# print(f"Final aggregated nutrition for {recipe_name}: {aggregated_nutrition}")

calculated_nutrition.append(aggregated_nutrition)


Ingredient: vöröshagyma, Proportion_100g: 2.0, Nutrition: {'Elsődleges kategória': 0, 'Másodlagos kategória': 0, 'Fehérje': 2.2, 'Zsír': 0.144, 'Szénhidrát': 18.68, 'Víz': 178.22, 'Foszfor': 0.058, 'Magnézium': 0.02, 'Nátrium': 0.008, 'Kalcium': 0.046, 'B6 vitamin': 0.24, 'Kolin': 0.0121999999999998, 'Niacin - B3 vitamin': 0.000232, 'C vitamin': 0.0148, 'Telített zsírsav': 0.084, 'Többszörösen telített zsírsav': 0.034, 'Vas': 0.0004199999999998, 'Cink': 0.00034, 'Réz': 7.8e-05, 'Mangán': 0.000258, 'Szelén': 1.0, 'Rost': 3.4, 'Cukor': 8.48, 'Tiamin - B1 vitamin': 9.2e-05, 'Riboflavin - B2 vitamin': 5.4e-05, 'Pantoténsav - B5 vitamin': 0.000246, 'Folát': 38.0, 'E vitamin': 4e-05, 'D vitamin': 0, 'B12 vitamin': 0, 'Egyszeresen telített zsírsav': 0.026, 'Béta-karotin': 2.0, 'Lutein+zeaxantin': 8.0, 'K vitamin': 0.8, 'A vitamin': 0, 'Likopin': 0, 'Alfa-karotin': 0, 'Béta-kriptoxantin': 0, 'koleszterin': 0, 'Retinol': 0, 'Folsav': 0}
Ingredient: sárgarépa, Proportion_100g: 1, Nutrition: {'El

In [135]:
calculated_nutrition

[{'Elsődleges kategória': 0,
  'Másodlagos kategória': 0,
  'Fehérje': 156.63619999999997,
  'Zsír': 490656.3327800001,
  'Szénhidrát': 256.264,
  'Víz': 722.3854,
  'Foszfor': 1.5227200000000003,
  'Magnézium': 0.29742,
  'Nátrium': 1.06882,
  'Kalcium': 0.85194,
  'B6 vitamin': 2.39432,
  'Kolin': 0.33214999999999945,
  'Niacin - B3 vitamin': 0.049179139999999746,
  'C vitamin': 0.035536,
  'Telített zsírsav': 69.32646,
  'Többszörösen telített zsírsav': 20.577419999999996,
  'Vas': 0.015381999999999802,
  'Cink': 0.01154259999999985,
  'Réz': 0.0012126199999999998,
  'Mangán': 0.0032887999999998974,
  'Szelén': 232.86,
  'Rost': 17.631999999999998,
  'Cukor': 32.156000000000006,
  'Tiamin - B1 vitamin': 0.00297882,
  'Riboflavin - B2 vitamin': 0.0019238399999995499,
  'Pantoténsav - B5 vitamin': 0.00596367999999995,
  'Folát': 727.08,
  'E vitamin': 0.008375999999999849,
  'D vitamin': 50.5,
  'B12 vitamin': 2.197,
  'Egyszeresen telített zsírsav': 66.4289,
  'Béta-karotin': 8396.3,

In [36]:
# Running the function
df_receptek = calculate_recipe_nutrition(df_receptek, df_alapanyagok)

TypeError: can't multiply sequence by non-int of type 'float'